# Data reduction

The Artportalen Dataset is *large*. The purpose of this notebook is to remove information we do not need. To execute it, your working directory should contain a folder called ’data’, itself containing a CSV file called ’artportalen.csv’.

As a reminder, the artportalen dataset can be downloaded [here](https://www.gbif.org/dataset/38b4c89f-584c-41bb-bd8f-cd1def33e92f). It will be downloaded in the form of a ~15GB zip which will unzip into a ~60GB CSV. The name of the downloaded file will be generated when you will download it, so you should rename it to ’artportalen.csv’ one unzipped.

---

This notebook is *slow* (but a one time cost) and shouldn't be run multiple times if it works for you. Moreover, it will generate ~13GB of files. Once done, you can remove the original artportalen dataset from your disk.

In [1]:
from tqdm.notebook import tqdm # progress bars!

In [2]:
f = open('data/artportalen.csv', 'r')
header = f.readline().replace("\n", '').split("\t")
f.close()

for i, column in enumerate(header):
    print(i, column)

0 gbifID
1 datasetKey
2 occurrenceID
3 kingdom
4 phylum
5 class
6 order
7 family
8 genus
9 species
10 infraspecificEpithet
11 taxonRank
12 scientificName
13 verbatimScientificName
14 verbatimScientificNameAuthorship
15 countryCode
16 locality
17 stateProvince
18 occurrenceStatus
19 individualCount
20 publishingOrgKey
21 decimalLatitude
22 decimalLongitude
23 coordinateUncertaintyInMeters
24 coordinatePrecision
25 elevation
26 elevationAccuracy
27 depth
28 depthAccuracy
29 eventDate
30 day
31 month
32 year
33 taxonKey
34 speciesKey
35 basisOfRecord
36 institutionCode
37 collectionCode
38 catalogNumber
39 recordNumber
40 identifiedBy
41 dateIdentified
42 license
43 rightsHolder
44 recordedBy
45 typeStatus
46 establishmentMeans
47 lastInterpreted
48 mediaType
49 issue


Clearly we are not interested in every column. Even some useful column could/should be removed (as long as we can reconstruct the information) to produce a more manageable file size/
 - Included in the data folder is a ’species.csv’ file, which allows for the reconstruction of columns 3 to 13 only from column 33. I suggest we trim those columns except for 12 (for human readability, for now) and only keep column 33. 
 - Location and date data should be kept, they are at the center of the project
 - Locality and region may get useful to ponder the number of observation by population (although I am not sure it is a good idea). UPDATE : after examining the ’locality’ column, it is disgusting to the point of being unusable. 

In the end, once the CSV read, we keep the entries at the following indices

In [3]:
kept_indices = [12, 33, 11, 17, 21, 22, 23, 30, 31, 32]
print([header[i] for i in kept_indices])

['scientificName', 'taxonKey', 'taxonRank', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters', 'day', 'month', 'year']


In [4]:
count_format = 0 # to count the number of imcomplete rows we discard
count_na = 0 # to count the number of rows with missing values
nb_lines = 108_073_381 # number of rows - SHOULD BE UPDATED FOR NEWER VERSIONS OF THE DATASET

f = open("data/artportalen.csv", "r")
out = open("data/artportalen_trimmed.csv", "w")

# Annoyingly, the fields contain some usual separators, such as ',', '\t' and even ';'.
# We use "|" anyway, because its presence in a field indicates an issue anyway.
# We will store the lines that contain a ";" in a the following in a separate file, and treat them separately.
# CAVEAT: if the ";" is in a field after the 33rd, we will not catch it, as it does not cause alignment issues.
out_badly_formatted = open("data/artportalen_badly_formatted.csv", "w")

f.readline() # skip the first line
out.write(";".join([header[i] for i in kept_indices]) + "\n")

for line in tqdm(f, total=nb_lines):

    line = line.replace("\n", '').split("\t")
    if any(";" in field for field in line[:33]):
        out_badly_formatted.write("|".join(line) + "\n")
        count_format += 1
        continue
    line = [str(line[k]) for k in kept_indices]

    # if all fields are not filled, we discard the line
    if not all(line):
        count_na += 1
        continue

    to_write = ";".join(line) + "\n"
    out.write(to_write)

f.close()
out.close()
out_badly_formatted.close()

count = count_na + count_format
print(f"Skipped {count} lines out of {nb_lines}, i.e. {count/nb_lines*100:.2f}%")
print(f"Skipped {count_na} lines because of missing values, and {count_format} lines because of formatting issues.")
print(f"Kept {nb_lines - count} lines.")

  0%|          | 0/108073382 [00:00<?, ?it/s]

Skipped 7940273 lines out of 108073382, i.e. 7.35%
Skipped 7895663 lines because of missing values, and 44610 lines because of formatting issues.
Kept 100133109 lines.


In the following, we select only the rows that contain observations of arthropods, from the trimmed data we have generated. Since we have removed the explicit mention of that information, we will have to recover it from the ’species.csv’ dataset.

In [7]:
f = open("data/artportalen_trimmed.csv", "r")
out = open("data/artportalen_arthropods.csv", "w")

count = 0 # to count the number of arthropods

f.readline() # skip header
out.write(";".join([header[i] for i in kept_indices]) + "\n")

for line in tqdm(f, total = 100133108):
    taxonKey = int(line.split(";")[1])
    if taxonKey in arthropods_keys:
        out.write(line)
        count += 1
f.close()
out.close()

print(f"Wrote {count} lines in the new file.")

  0%|          | 0/100133108 [00:00<?, ?it/s]

Wrote 6406801 lines in the new file.


In [8]:
arthro_df = pd.read_csv("data/artportalen_arthropods.csv", sep=";")
arthro_df.info()

The ’artportalen_arthropods.csv’ is small enough to be held in memory (~800MB) but still unwieldy. In the following, we replace the text entries by more compact datatypes. First, we build dictionnaries for ’taxonRank, locality, stateProvince’. We do so directly from the artportalen_trimmed dataset to future proof the construction. We also lose the ’scientificName’ in favor of the taxonKey.

In [10]:
f = open("data/artportalen_trimmed.csv", "r")
rank_dict = {}
prov_dict = {}

f.readline() # skip header

for line in tqdm(f, total = 100133108):
    rank, prov = line.split(";")[2:4]
    if rank not in rank_dict:
        rank_dict[rank] = len(rank_dict) + 1
    if prov not in prov_dict:
        prov_dict[prov] = len(prov_dict) + 1

f.close()

  0%|          | 0/100133108 [00:00<?, ?it/s]

In [15]:
f = open("data/artportalen_arthropods.csv", "r")
out = open("data/arthropods_compressed.csv", "w")

f.readline() # skip header
out.write(";".join([header[i] for i in kept_indices][1:]) + "\n")

for line in tqdm(f, total = 6406801):
    line = line.split(";")
    line[2] = str(rank_dict[line[2]])
    line[3] = str(prov_dict[line[3]])
    out.write(";".join(line[1:]))

f.close()
out.close()

  0%|          | 0/6406801 [00:00<?, ?it/s]

We also need to remember the inverse mapping.

In [13]:
key_to_rank = open("data/ranks.csv", "w")
for k, v in rank_dict.items():
    key_to_rank.write(f"{v};{k}\n")
key_to_rank.close()

key_to_prov = open("data/provinces.csv", "w")
for k, v in prov_dict.items():
    key_to_prov.write(f"{v};{k}\n")
key_to_prov.close()

In [36]:
import pandas as pd
arthro_df = pd.read_csv("data/arthropods_compressed.csv", sep=";")
arthro_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6406801 entries, 0 to 6406800
Data columns (total 9 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   taxonKey                       int64  
 1   taxonRank                      int64  
 2   stateProvince                  int64  
 3   decimalLatitude                float64
 4   decimalLongitude               float64
 5   coordinateUncertaintyInMeters  float64
 6   day                            int64  
 7   month                          int64  
 8   year                           int64  
dtypes: float64(3), int64(6)
memory usage: 439.9 MB


In [47]:
arthro_df.describe()

,taxonKey,taxonRank,stateProvince,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,day,month,year
count,6.406801e+06,6.406801e+06,6.406801e+06,6.406801e+06,6.406801e+06,6.406801e+06,6.406801e+06,6.406801e+06,6.406801e+06
mean,3.514375e+06,1.080315e+00,1.043600e+01,5.851040e+01,1.554494e+01,2.480183e+02,1.570498e+01,6.616672e+00,2.012136e+03
std,2.310070e+06,5.773583e-01,5.910404e+00,2.391516e+00,2.272081e+00,2.662950e+03,8.857103e+00,1.610713e+00,1.497431e+01
min,5.400000e+01,1.000000e+00,1.000000e+00,5.533700e+01,1.006286e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.749000e+03
25%,1.773163e+06,1.000000e+00,6.000000e+00,5.667865e+01,1.356081e+01,5.000000e+01,8.000000e+00,6.000000e+00,2.009000e+03
50%,1.986171e+06,1.000000e+00,1.000000e+01,5.810863e+01,1.562799e+01,1.000000e+02,1.600000e+01,7.000000e+00,2.017000e+03
75%,5.086570e+06,1.000000e+00,1.500000e+01,5.958938e+01,1.716239e+01,2.500000e+02,2.300000e+01,8.000000e+00,2.021000e+03
max,1.238534e+07,9.000000e+00,3.400000e+01,6.905934e+01,2.416028e+01,6.400798e+06,3.100000e+01,1.200000e+01,2.024000e+03


Clearly ’coordinateUncertaintyInMeters’ contains some nonsensical values. How many values are above 5km ?

In [58]:
prop = len(arthro_df[arthro_df["coordinateUncertaintyInMeters"] > 5000])/len(arthro_df)*100
print(f"A negligible proportion ({prop:.2f}%) of rows have an uncertainty greater than 5km. Let's remove them.")
arthro_df.drop(arthro_df[arthro_df["coordinateUncertaintyInMeters"] > 5000].index, inplace=True)
arthro_df["coordinateUncertaintyInMeters"].describe().astype(int)

A negligible proportion (0.00%) of rows have an uncertainty greater than 5km. Let's remove them.


count    6405697
mean         245
std          550
min            1
25%           50
50%          100
75%          250
max         5000
Name: coordinateUncertaintyInMeters, dtype: int64

In [63]:
arthro_df["coordinateUncertaintyInMeters"].quantile([0.9 + 0.01*k for k in range(1, 11)])

0.91     500.0
0.92     500.0
0.93     750.0
0.94    1000.0
0.95    1000.0
0.96    1000.0
0.97    1000.0
0.98    1500.0
0.99    2500.0
1.00    5000.0
Name: coordinateUncertaintyInMeters, dtype: float64

In the remainder of the data, 